In [21]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MultiLabelBinarizer
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report, accuracy_score

# Load the dataset
df = pd.read_excel('../../Godzilla.xlsx')

# Convert the 'Movie Budget (Yen)' and 'Final Revenue (Yen)' columns to numeric values, removing any non-numeric characters
df['Movie Budget (Yen)'] = df['Movie Budget (Yen)'].replace('[\¥,\,]', '', regex=True).replace('-', np.nan).astype(float)
df['Final Revenue (Yen)'] = df['Final Revenue (Yen)'].replace('[\¥,\,]', '', regex=True).replace('-', np.nan).astype(float)

# Convert Yen to Dollars (1 USD = 140 JPY)
conversion_rate = 140
df['Movie Budget (USD)'] = df['Movie Budget (Yen)'] / conversion_rate
df['Final Revenue (USD)'] = df['Final Revenue (Yen)'] / conversion_rate

# Drop the Yen columns as we are using the USD columns for modeling
df = df.drop(columns=['Movie Budget (Yen)', 'Final Revenue (Yen)', 'Position', 'URL', 'Japanese Title', 'Godzilla\'s Sizes (In Meters)'])

# Multi-label binarizer for genres
mlb = MultiLabelBinarizer()
df['Genres'] = df['Genres'].apply(lambda x: x.split(', '))  # assuming genres are separated by ', '
genres = mlb.fit_transform(df['Genres'])

# Create a DataFrame for genres
genre_df = pd.DataFrame(genres, columns=mlb.classes_)

# Drop original 'Genres' column and concatenate the new genre columns
df = df.drop(columns=['Genres'])
df = pd.concat([df, genre_df], axis=1)

# Select relevant features for modeling
features = ['Movie Budget (USD)', 'Runtime (mins)', "Godzilla's Sizes (In Feet)", 'IMDb Rating', 'Rotten Tomatoes Rating', 'Year']
target = mlb.classes_

In [22]:
# Split the data into training and testing sets
X = df[features]
y = genre_df

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define transformers for numerical features
numerical_features = ['Movie Budget (USD)', 'Runtime (mins)', "Godzilla's Sizes (In Feet)", 'IMDb Rating', 'Rotten Tomatoes Rating', 'Year']

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Fill missing values with median
    ('scaler', StandardScaler())  # Scale numerical values
])

# Combine transformers into a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),  # Apply numerical transformer to numerical features
    ])

In [23]:
# Preprocess the data
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

# Train the model
model = MultiOutputClassifier(RandomForestClassifier(n_estimators=300, random_state=42))
model.fit(X_train, y_train)

MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=300,
                                                       random_state=42))

In [25]:
# Save the trained model and the preprocessor
with open('godzilla_multilabel_model.pkl', 'wb') as f:
    pickle.dump(model, f)
with open('preprocessor.pkl', 'wb') as f:
    pickle.dump(preprocessor, f)
with open('mlb.pkl', 'wb') as f:
    pickle.dump(mlb, f)

In [24]:
# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
for i, genre in enumerate(target):
    print(f"Classification Report for genre {genre}:\n", classification_report(y_test.iloc[:, i], y_pred[:, i]))
    print(f"Accuracy Score for genre {genre}:", accuracy_score(y_test.iloc[:, i], y_pred[:, i]))

print("\nOverall Accuracy Score:", accuracy_score(y_test, y_pred))

Classification Report for genre Action:
               precision    recall  f1-score   support

           1       1.00      1.00      1.00         8

    accuracy                           1.00         8
   macro avg       1.00      1.00      1.00         8
weighted avg       1.00      1.00      1.00         8

Accuracy Score for genre Action: 1.0
Classification Report for genre Adventure:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.38      1.00      0.55         3

    accuracy                           0.38         8
   macro avg       0.19      0.50      0.27         8
weighted avg       0.14      0.38      0.20         8

Accuracy Score for genre Adventure: 0.375
Classification Report for genre Animation:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         7
           1       1.00      1.00      1.00         1

    accuracy                

c:\Users\Lardex\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Lardex\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Lardex\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Lardex\anaconda3\Lib\site-packages\sklearn\

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   English Title               36 non-null     object        
 1   IMDb Rating                 36 non-null     float64       
 2   Rotten Tomatoes Rating      36 non-null     float64       
 3   Runtime (mins)              36 non-null     int64         
 4   Year                        36 non-null     int64         
 5   Num Votes                   36 non-null     int64         
 6   Release Date                36 non-null     datetime64[ns]
 7   Directors                   36 non-null     object        
 8   Godzilla's Sizes (In Feet)  36 non-null     int64         
 9   Era                         36 non-null     object        
 10  Number of Monsters          36 non-null     int64         
 11  Tickets Sold                36 non-null     int64         
 

In [13]:
df.head()

,English Title,IMDb Rating,Rotten Tomatoes Rating,Runtime (mins),Year,Genres,Num Votes,Release Date,Directors,Godzilla's Sizes (In Feet),Era,Number of Monsters,Tickets Sold,Movie Budget (USD),Final Revenue (USD),Release Year
0,Godzilla (1954),7.5,0.93,96,1954,"Horror, Sci-Fi",33861,1954-11-03,Ishirô Honda,164,Showa,1,9610000,7.214286e+05,1.307143e+06,1954
1,Godzilla Raids Again,5.8,0.60,82,1955,"Action, Horror, Sci-Fi",6922,1955-04-24,"Ishirô Honda, Motoyoshi Oda",164,Showa,2,8340000,7.142857e+05,1.214286e+06,1955
2,King Kong vs. Godzilla,5.7,0.50,97,1962,"Action, Adventure, Fantasy, Sci-Fi",10398,1963-06-03,"Tom Montgomery, Ishirô Honda",164,Showa,1,11200000,1.071429e+06,3.071429e+06,1963
3,Mothra vs. Godzilla,6.5,0.92,89,1964,"Adventure, Fantasy, Sci-Fi",8934,1964-04-29,Ishirô Honda,164,Showa,2,7220000,8.785714e+05,4.464286e+06,1964
4,Ghidorah: The Three-Headed Monster,6.5,0.75,93,1964,"Action, Adventure, Fantasy, Sci-Fi, Thriller",6703,1964-12-20,Ishirô Honda,164,Showa,4,5410000,NaN,2.785714e+06,1964
